In [ ]:
# Workaround to install pystruct.
!pip install --upgrade pip setuptools==57.5.0 
!git clone https://github.com/pystruct/pystruct && cd pystruct
!curl https://patch-diff.githubusercontent.com/raw/pystruct/pystruct/pull/221.patch | git apply
!pip install pystruct/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 14.9 MB/s 
     |████████████████████████████████| 819 kB 58.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Cloning into 'pystruct'...
remote: Enumerating objects: 6974, done.
remote: Total 6974 (delta 0), reused 0 (delta 0), pack-reused 6974
Receiving objects: 100% (6974/6974), 12.16 MiB | 12.64 MiB/s, done.
Resolving deltas: 100% (4357/4357), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1260k    0 1260k    0     0  1404k      0 --:--:-- --:--:-- --:--:-- 1403k
<stdin>:912: trailing whitespace.
 * 
<stdin>:915: trailing whitespace.
 * 
<stdin>:2642: trailing whitespace.
      __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg, __pyx_v_ndarray); 
<stdin>:2652: trailing whitespace.
      __pyx_t_2 = __pyx_memoryview_check(__pyx_v_arg); 
<stdin>:2660: trailing whitespace.
        __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg_base, __pyx_v_ndarray); 
error: src/utils.c: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simpl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_image_folder_path = 'drive/MyDrive/btp-dataset/Train/Train/Image/'
train_annotation_folder_path = 'drive/MyDrive/btp-dataset/Train/Train/Annotations/'
train_prepared_annotation_folder_path = 'drive/MyDrive/btp-dataset/Train/Prepared/Annotations/'
testing_image_folder_path = 'drive/MyDrive/btp-dataset/Train/Train/Image/'
testing_annotation_folder_path = 'drive/MyDrive/btp-dataset/Train/Train/Annotations/'
saved_unary_model_path =  'drive/MyDrive/btp-dataset/SavedModels/unary/'
saved_pairwise_model_path =  'drive/MyDrive/btp-dataset/SavedModels/pairwise/'
saved_crf_model_path =  'drive/MyDrive/btp-dataset/SavedModels/crf/'

In [ ]:
from tqdm.notebook import tqdm
import json
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.metrics import sparse_categorical_crossentropy, categorical_crossentropy
from pystruct.utils import SaveLogger

In [ ]:
def get_unary_lines(human_read_lines, img):
  lines = []
  for line in human_read_lines:
    x1, y1, x2, y2 = line.x1, line.y1, line.x2, line.y2
    scaled_line_img = cv2.resize(img[y1:y2+1, x1:x2+1], (640, 32))
    lines.append(scaled_line_img)

  return lines

In [ ]:
def get_pairwise_lines(human_read_lines, img):
  lines = []
  for i in range(len(human_read_lines)-1):
    y11, x11 = human_read_lines[i].y1, human_read_lines[i].x1
    y12, x12 = human_read_lines[i].y2, human_read_lines[i].x2
    y21, x21 = human_read_lines[i+1].y1, human_read_lines[i+1].x1
    y22, x22 = human_read_lines[i+1].y2, human_read_lines[i+1].x2

    scaled_img1 = cv2.resize(img[y11:y12+1, x11:x12+1], (640, 16))
    scaled_img2 = cv2.resize(img[y21:y22+1, x21:x22+1], (640, 16))
    scaled_line_img = cv2.vconcat([scaled_img1, scaled_img2])
    lines.append(scaled_line_img)
  return lines

In [ ]:
def scale_img_for_unary(img, x1, x2, y1, y2):
    
    width = x2-x1+1
    height = y2-y1+1

    final_img = img[y1:y2+1, x1:x2+1]

    if (width <= 640 and height <= 32):
      vertical_padding = 32 - height
      horizontal_padding = 640 - width
      final_img = cv2.copyMakeBorder(
          final_img, 
          vertical_padding//2,
          vertical_padding//2 + vertical_padding%2,
          horizontal_padding//2,
          horizontal_padding//2 + horizontal_padding%2,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
        )

    elif (height <= 32):
      scale_down = width/640
      adjusted_height = int(height/scale_down)
      final_img = cv2.resize(final_img, (640, adjusted_height))

      vertical_padding = 32 - adjusted_height

      final_img = cv2.copyMakeBorder(
          final_img, 
          vertical_padding//2,
          vertical_padding//2 + vertical_padding%2,
          0,
          0,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
      )

    elif (width <= 640):
      scale_down = height/32
      adjusted_width = int(width/scale_down)
      final_img = cv2.resize(final_img, (adjusted_width, 32))

      horizontal_padding = 640 - adjusted_width

      final_img = cv2.copyMakeBorder(
          final_img, 
          0,
          0,
          horizontal_padding//2,
          horizontal_padding//2 + horizontal_padding%2,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
      )

    else:
      scale_down_by_width = width/640
      scale_down_by_height = height/32

      if (scale_down_by_height < scale_down_by_width):
        adjusted_height = int(height/scale_down_by_width)
        final_img = cv2.resize(final_img, (640, adjusted_height))
        vertical_padding = 32 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img, 
            vertical_padding//2,
            vertical_padding//2 + vertical_padding%2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255)
        )

      else:
        adjusted_width = int(width/scale_down_by_height)
        final_img = cv2.resize(final_img, (adjusted_width, 32))
        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img, 
            0,
            0,
            horizontal_padding//2,
            horizontal_padding//2 + horizontal_padding%2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255)
        )
    
    return final_img

In [ ]:
def scale_img_for_pairwise(img, x1, x2, y1, y2):
    
    width = x2-x1+1
    height = y2-y1+1

    final_img = img[y1:y2+1, x1:x2+1]

    if (width <= 640 and height <= 16):
      vertical_padding = 16 - height
      horizontal_padding = 640 - width
      final_img = cv2.copyMakeBorder(
          final_img, 
          vertical_padding//2,
          vertical_padding//2 + vertical_padding%2,
          horizontal_padding//2,
          horizontal_padding//2 + horizontal_padding%2,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
        )

    elif (height <= 16):
      scale_down = width/640
      adjusted_height = int(height/scale_down)
      final_img = cv2.resize(final_img, (640, adjusted_height))

      vertical_padding = 16 - adjusted_height

      final_img = cv2.copyMakeBorder(
          final_img, 
          vertical_padding//2,
          vertical_padding//2 + vertical_padding%2,
          0,
          0,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
      )

    elif (width <= 640):
      scale_down = height/16
      adjusted_width = int(width/scale_down)
      final_img = cv2.resize(final_img, (adjusted_width, 16))

      horizontal_padding = 640 - adjusted_width

      final_img = cv2.copyMakeBorder(
          final_img, 
          0,
          0,
          horizontal_padding//2,
          horizontal_padding//2 + horizontal_padding%2,
          cv2.BORDER_CONSTANT,
          None,
          (255, 255, 255)
      )

    else:
      scale_down_by_width = width/640
      scale_down_by_height = height/16

      if (scale_down_by_height < scale_down_by_width):
        adjusted_height = int(height/scale_down_by_width)
        final_img = cv2.resize(final_img, (640, adjusted_height))
        vertical_padding = 16 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img, 
            vertical_padding//2,
            vertical_padding//2 + vertical_padding%2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255)
        )

      else:
        adjusted_width = int(width/scale_down_by_height)
        final_img = cv2.resize(final_img, (adjusted_width, 16))
        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img, 
            0,
            0,
            horizontal_padding//2,
            horizontal_padding//2 + horizontal_padding%2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255)
        )
    
    return final_img

In [ ]:
def get_unary_class(u):
  if (u == 'figureRegion'):
    return 0
  elif (u == 'tableRegion'):
    return 1
  elif (u == 'formulaRegion'):
    return 2
  else:
    return 3

def get_unary_data_for_page(annotation_filename, cnn=False):
  train_samples = []
  train_labels = []

  image_filename = annotation_filename[:-4]+'bmp'
  img = cv2.imread(train_image_folder_path+image_filename)

  with open(train_prepared_annotation_folder_path+annotation_filename, 'r') as f:
    data = json.load(f)

    for line in data['lines']:
      try:
        x1, y1, x2, y2 = line['x1'], line['y1'], line['x2'], line['y2']
        scaled_line_img = scale_img_for_unary(img, x1, x2, y1, y2)
        train_samples.append(scaled_line_img)
        train_labels.append(get_unary_class(line['block_class']))
        
        if (cnn and  get_unary_class(line['block_class']) != get_unary_class('textRegion')):
          train_samples.append(cv2.flip(scaled_line_img, 0))
          train_labels.append(get_unary_class(line['block_class']))
          train_samples.append(cv2.flip(scaled_line_img, 1))
          train_labels.append(get_unary_class(line['block_class']))
          train_samples.append(cv2.flip(scaled_line_img, -1))
          train_labels.append(get_unary_class(line['block_class']))
      except Exception as e:
        print(e)
        pass

  return train_samples, train_labels

In [ ]:
def get_pairwise_class(u,v):
  if (v == 'textRegion' and u == 'tableRegion'):
    return 0
  elif (v == 'textRegion' and u == 'formulaRegion'):
    return 1
  elif (v == 'textRegion' and u == 'textRegion'):
    return 2
  elif (v == 'textRegion' and u == 'figureRegion'):
    return 3
  elif (v == 'tableRegion' and u == 'tableRegion'):
    return 4
  elif (v == 'tableRegion' and u == 'formulaRegion'):
    return 5
  elif (v == 'tableRegion' and u == 'textRegion'):
    return 6
  elif (v == 'tableRegion' and u == 'figureRegion'):
    return 7
  elif (v == 'figureRegion' and u == 'tableRegion'):
    return 8
  elif (v == 'figureRegion' and u == 'formulaRegion'):
    return 9
  elif (v == 'figureRegion' and u == 'textRegion'):
    return 10
  elif (v == 'figureRegion' and u == 'figureRegion'):
    return 11
  elif (v == 'formulaRegion' and u == 'tableRegion'):
    return 12
  elif (v == 'formulaRegion' and u == 'formulaRegion'):
    return 13
  elif (v == 'formulaRegion' and u == 'textRegion'):
    return 14
  elif (v == 'formulaRegion' and u == 'figureRegion'):
    return 15
  else:
    return -1

def get_pairwise_data_for_page(annotation_filename):
  train_samples = []
  train_labels = []
  image_filename = annotation_filename[:-4]+'bmp'
  img = cv2.imread(train_image_folder_path+image_filename)

  with open(train_prepared_annotation_folder_path+annotation_filename, 'r') as f:
    data = json.load(f)
    line_list = data['lines']

    for i in range(len(line_list)-1):
      try:
        y11, x11 = line_list[i]['y1'], line_list[i]['x1']
        y12, x12 = line_list[i]['y2'], line_list[i]['x2']
        y21, x21 = line_list[i+1]['y1'], line_list[i+1]['x1']
        y22, x22 = line_list[i+1]['y2'], line_list[i+1]['x2']
        scaled_img1 = scale_img_for_pairwise(img, x11, x12, y11, y12)
        scaled_img = scale_img_for_pairwise(img, x21, x22, y21, y22)
        scaled_img1 = cv2.resize(img[y11:y12+1, x11:x12+1], (640, 16))
        scaled_img2 = cv2.resize(img[y21:y22+1, x21:x22+1], (640, 16))
        scaled_line_img = cv2.vconcat([scaled_img1, scaled_img2])

        train_samples.append(scaled_line_img)
        train_labels.append(get_pairwise_class(line_list[i]['block_class'], line_list[i+1]['block_class']))
      except Exception as e:
        pass
  return train_samples, train_labels

In [ ]:
def get_unary_cnn_training_data():
  print('fetching unary data...')
  train_samples = []
  train_labels = []
  file_counter = 0

  list_of_files = os.listdir(train_prepared_annotation_folder_path)
  total_number_of_files = len(list_of_files)

  with tqdm(total=total_number_of_files) as pbar:

    for annotation_filename in list_of_files:
      file_counter += 1
      x_train, y_train = get_unary_data_for_page(annotation_filename, True)
      train_samples.extend(x_train)
      train_labels.extend(y_train)
      pbar.update(1)
  train_samples = np.array(train_samples)
  train_labels = np.array(train_labels)
      
  return train_samples, train_labels

In [ ]:
def get_pairwise_cnn_training_data():
  print('fetching pairwise data...')
  train_samples = []
  train_labels = []
  file_counter = 0

  list_of_files = os.listdir(train_prepared_annotation_folder_path)
  total_number_of_files = len(list_of_files)

  with tqdm(total=total_number_of_files) as pbar:

    for annotation_filename in list_of_files:
      file_counter += 1
      x_train, y_train = get_pairwise_data_for_page(annotation_filename)
      train_samples.extend(x_train)
      train_labels.extend(y_train)
      pbar.update(1)
  train_samples = np.array(train_samples)
  train_labels = np.array(train_labels)

  return train_samples, train_labels

In [ ]:
unary = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 640, 3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(4, activation='relu')
])

unary.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 638, 32)       896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 636, 32)       9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 318, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 316, 64)       18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 314, 64)       36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 157, 64)       0         
 2D)                                                    

In [ ]:
pairwise = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape= (32,640,3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(16, activation='relu')
])
pairwise.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 638, 32)       896       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 636, 32)       9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 318, 32)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 316, 64)       18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 10, 314, 64)       36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 157, 64)       0         
 2D)                                                  

In [ ]:
unary.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   

In [ ]:
pairwise.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train_u, y_train_u = get_unary_cnn_training_data()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
unary.fit(x_train_u, y_train_u, validation_split = 0.2, batch_size=64, epochs=30, shuffle=False, verbose=1, callbacks = [callback])
unary.save(saved_unary_model_path)

fetching unary data...


  0%|          | 0/1600 [00:00<?, ?it/s]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertio

In [ ]:
x_train_p, y_train_p = get_pairwise_cnn_training_data()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
pairwise.fit(x_train_p, y_train_p, validation_split = 0.2, batch_size=64, epochs=30, shuffle=False, verbose=1, callbacks = [callback])
pairwise.save(saved_pairwise_model_path)

ModuleNotFoundError: ignored